In [52]:
!pip install pyspark
!pip install findspark

In [122]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Credit Card Fraud Detection").getOrCreate()
data = spark.read.csv("/content/creditcard.csv", header=True, inferSchema=True)
data.show()

+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|                V20|                 V

In [123]:
data.printSchema()

root
 |-- Time: integer (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (null

In [124]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [57]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

feature_cols = ["Amount", "Time"] + [f"V{i}" for i in range(1, 29)]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

rf = RandomForestClassifier(labelCol="Class", featuresCol="features")

pipeline = Pipeline(stages=[assembler, rf])


In [76]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [77]:
# Create a list of categorical features
categFeatures = ["Amount", "Time"]

# Encode categorical string cols to label indices
stringIndexers = [StringIndexer().setInputCol(baseFeature).setOutputCol(baseFeature + "_idx") for baseFeature in categFeatures]

# Convert categorical label indices to binary vectors
encoders = [OneHotEncoder().setInputCol(baseFeature + "_idx").setOutputCol(baseFeature + "_vec") for baseFeature in categFeatures]

# Combine all columns into a single feature vector
featureAssembler = VectorAssembler()
featureAssembler.setInputCols([baseFeature + "_vec" for baseFeature in categFeatures])
featureAssembler.setOutputCol("features")

VectorAssembler_94c2cc50f443

In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('/content/creditcard.csv')

feature_columns = data.columns.difference(['target_column'])

X = data[["Amount", "Time"]]
y = data['Class']

X = X.dropna()
y = y[y.index.isin(X.index)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


print(f"Accuracy: {accuracy*100:.2f}%")


Confusion Matrix:
 [[5236   12]
 [  16    3]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5248
         1.0       0.20      0.16      0.18        19

    accuracy                           0.99      5267
   macro avg       0.60      0.58      0.59      5267
weighted avg       0.99      0.99      0.99      5267

Accuracy: 99.47%


In [112]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Load your dataset
data = pd.read_csv("/content/creditcard.csv")

# Drop rows with missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop("Class", axis=1)
y = data["Class"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100:.2f}%")
print()
print('-----------------------------------------------------')


Confusion Matrix:
 [[7867    5]
 [  11   17]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7872
         1.0       0.77      0.61      0.68        28

    accuracy                           1.00      7900
   macro avg       0.89      0.80      0.84      7900
weighted avg       1.00      1.00      1.00      7900

Accuracy: 99.80%

-----------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [117]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC  # Support Vector Machine classifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load your dataset
data = pd.read_csv("/content/creditcard.csv")

# Drop rows with missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop("Class", axis=1)
y = data["Class"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train a Support Vector Machine (SVM) classifier
model = SVC()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100:.2f}%")
print()
print('-----------------------------------------------------')


Confusion Matrix:
 [[7871    1]
 [  13   15]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7872
         1.0       0.94      0.54      0.68        28

    accuracy                           1.00      7900
   macro avg       0.97      0.77      0.84      7900
weighted avg       1.00      1.00      1.00      7900

Accuracy: 99.82%

-----------------------------------------------------


In [118]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load your dataset
data = pd.read_csv("/content/creditcard.csv")

# Drop rows with missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop("Class", axis=1)
y = data["Class"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train a Random Forest classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100:.2f}%")
print()
print('-----------------------------------------------------')


Confusion Matrix:
 [[7871    1]
 [   7   21]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7872
         1.0       0.95      0.75      0.84        28

    accuracy                           1.00      7900
   macro avg       0.98      0.87      0.92      7900
weighted avg       1.00      1.00      1.00      7900

Accuracy: 99.90%

-----------------------------------------------------


In [119]:
spark.stop()

*Random forest algorithm gives best output*